<a href="https://colab.research.google.com/github/almedida/thesis/blob/main/simulation_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
rm(list = ls())

##**Install Packages**##

In [ ]:
install.packages('pacman')
library(pacman, devtools)
p_load("tidyverse", "matrixTests", "gtools", "VennDiagram", "tmvtnorm")

#install BiocManager
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

#install limma and qvalue packages
BiocManager::install(c("limma", "qvalue"))
library(limma)
library(qvalue)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.2 (2021-11-01)

Warning message:
“package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'limma' 'qvalue'”
Old packages: 'cli', 'clipr', 'colorspace', 'crayon', 'evaluate', 'glue',
  'jsonlite', 'openssl', 'rmarkdown', 'tidyselect', 'tinytex', 'xfun', 'yaml',
  'mgcv', 'survival'



##***Load Dataset***

In [ ]:
#load dataset
simulation_data = read.table("sim_data.csv", header= TRUE, sep=",")
N = ncol(simulation_data) #N = number columns (samples in the dataset)
n_gene = nrow(simulation_data) #n_gene = no of genes (rows in the dataset)

In [ ]:
# head(simulation_data)

In [ ]:
# dim(simulation_data)

10k randomly selected data for simulation process

In [ ]:
set.seed(1)
m = 10000 #no of genes to be selected from the dataset
randomly_selected_genes = sample(n_gene, m) #randomly selected genes of size m from the total genes

# head(randomly_selected_genes)
# length(randomly_selected_genes)

In [ ]:
#10000 selected data corresponding to the randomly selected genes (EE genes)
randomly_selected_data = simulation_data[randomly_selected_genes, ]
genej_sim_data = randomly_selected_data
# head(genej_sim_data)


In [ ]:
dim(genej_sim_data)

[1] 10000   217

In [ ]:
write.csv(genej_sim_data, "genej_sim_data.csv")

standarad deviation for 10000 genes from all experimental units

In [ ]:
genej_sim_data = read.csv("genej_sim_data.csv", header = TRUE, sep = ",")

## **Simulation**

In [ ]:
simulation = function(n, m11_genes){

  N = ncol(genej_sim_data) #N = number columns (samples in the dataset)

  genej_std = apply(genej_sim_data, 1, sd)  #calculate std for all genes

  #randomly select exp units, n=4
  n_samples = n #no of samples
  randomly_selected_samples  = sample(N, 2*n_samples) #randomly select samples of size 2*n_trt_grp 
                                                    #for treatment group
  treatment_group1 = randomly_selected_samples[1:n_samples]
  treatment_group2 = randomly_selected_samples[(n_samples + 1):(2*n_samples)]

  #group 2 data
  treatment_group2_data = genej_sim_data[treatment_group2]

  #generate treatment effects, j_effects=10000
  mu = genej_std
  treatment_effect_j = rnorm(10000, mu, genej_std)

  #add 10k treatments effects generated to group 2 data
  de_treatment_group2 = treatment_group2_data + treatment_effect_j

  n_de = m11_genes #number of m11
  n_treated_genes_grp2 = nrow(de_treatment_group2)

  #we extract m genes from DE_treatment_group2 (genes with treatment effects)
  de_genes_index = sample(n_treated_genes_grp2, n_de)

  #we copied untreated (genes without effects) genes from treatment_group2_data
  treated_genes_group2 = treatment_group2_data

  #we replaced the m untreated genes (without treatment effects) with the corresponding treated genes(with treatment effects)
  treated_genes_group2[de_genes_index, ] = de_treatment_group2[de_genes_index, ]



}

In [ ]:

genej_std = apply(genej_sim_data, 1, sd)
# head(genej_std)
# length(genej_std)

Here, we create 2 treatment groups with 4 randomly selected samples each with 10,000 randlomly selected genes.

In [ ]:
set.seed(Sys.time())
n_samples = 4 #no of samples
randomly_selected_samples  = sample(N, 2*n_samples) #randomly select samples of size 2*n_trt_grp 
                                                    #for treatment group
treatment_group1 = randomly_selected_samples[1:n_samples]
treatment_group2 = randomly_selected_samples[(n_samples + 1):(2*n_samples)]

Group 1 data

In [ ]:
treatment_group1_data = genej_sim_data[treatment_group1]
# head(treatment_group1_data)
# write.csv(treatment_group1_data, "10k_untreated_group15.csv")

* Group 1 consists of EE genes and group 2 consists of 9k EE genes and 1k DE genes (genes with treatment effects)
* treatment_group2_data consists of the 4 randomly selected samples (exp. units)

In [ ]:
treatment_group2_data = genej_sim_data[treatment_group2]
# dim(treatment_group2_data)
head(treatment_group2_data, 2)

,GSM1247303,GSM1247226,GSM1247341,GSM1247319
,<dbl>,<dbl>,<dbl>,<dbl>
17401,7.272378,7.501630,7.198761,6.606291
4775,3.996014,4.117879,3.772752,4.407703


**Simulation from 3.1**

We calculate the treatment effect by extracting the effect size of each gene j from a random normal distribution using genej_std (standard deviation for the 10k randomly selected genes from all experimental units in the population data)

In [ ]:
set.seed(1)
mu = genej_std

treatment_effect_j = rnorm(10000, mu, genej_std)
# length(treatment_effect_j)
#head(data.frame(treatment_effect_j))



**DE genes**

DE genes are created by adding treatment effects extracted to each gene in group 2

In [ ]:
dim(treatment_group2_data)
de_treatment_group2 = treatment_group2_data + treatment_effect_j
# head(de_treatment_group2)

[1] 10000     4

## we randomly selected 3000 genes with treatment effect from de_treatment_group2

In [ ]:
n_de = 5
n_treated_genes_grp2 = nrow(de_treatment_group2)

#we extract 3000 genes from DE_treatment_group2 (genes with treatment effects)
de_genes_index = sample(n_treated_genes_grp2, n_de)

#we copied untreated (genes without effects) genes from treatment_group2_data
treated_genes_group2 = treatment_group2_data

#we replaced the 3000 untreated genes (without treatment effects) with the corresponding treated genes(with treatment effects)
treated_genes_group2[de_genes_index, ] = de_treatment_group2[de_genes_index, ]

# dim(treated_genes_group2)
head(de_genes_index)

[1] 1017 8004 4775 9725 8462

## **Summary of Group 2 simulation** 

m00 - number of EE genes

m11 - number of DE genes


In [ ]:
m00 = treated_genes_group2[-de_genes_index, ]
m11 = treated_genes_group2[de_genes_index, ]
treatment_effect_size_grp2 = treatment_effect_j[de_genes_index]
# write.csv(treated_genes_group2, "10k_treated_group25.csv")
#dim(m00)
#dim(m11)
# length(treatment_effect_size_grp2)

## **Student t-test**

In [ ]:
group1_data = treatment_group1_data
group2_data = treated_genes_group2

In [ ]:
ttest <- row_t_equalvar(group1_data, group2_data, alternative = "two.sided", mu = 0, conf.level = 0.95)

In [ ]:
pvalue <- ttest[13]
# hist_data <- data.matrix(pvalue)
# hist(hist_data)

**Adjusted p-values using Benjamini Hochberg**

In [ ]:
adj_pvalue <- p.adjust(data.matrix(pvalue), "BH")
adj_pvalue_df <-  data.frame(adj_pvalue)

## **Limma t-test**

In [ ]:
limma_gene_exp <- c(group1_data, group2_data)
limma_gene_exp <- data.frame(limma_gene_exp)

**Create expression list**

create a factor list for the differentially expressed genes with group1 set as first level

In [ ]:
expr_list_10k <- factor(
  x = c(rep("group1",4), rep("group2",4)),
  levels=c("group1","group2")            # Set group 1 to be the first level
)

design <- model.matrix(~expr_list_10k)          # Remove the zero
# head(design)

In [ ]:
# #Now we can run the differential expression pipeline

fit <- lmFit(limma_gene_exp, design)
fit <- eBayes(fit)
results <- decideTests(fit)
# write.csv(summary(results), "summary_10k_genes_expr_sim5.csv")

In [ ]:
limma_pvalue <- fit$p.value[, 2]
limma_pvalue <- data.frame(limma_pvalue)


In [ ]:
adj_limma_pvalue <- data.matrix(limma_pvalue)
adj_limma_pvalue <- p.adjust(adj_limma_pvalue, "BH")
adj_limma_pvalue <- data.frame(adj_limma_pvalue)

## **Proposed Method**

In [ ]:
pval_raw <- c(pvalue, limma_pvalue)
pval_raw <- data.frame(pval_raw)
pvals1 <- (as.matrix(pval_raw[,1]))
pvals2 <- (as.matrix(pval_raw[,2]))

**Calculate cut-off points using Histogram-based method**

calc.cutoff slightly modifies the estimate.m0 function to return the appropriate p-value cutoff where all p-values greater than the cutoff are assumed to come from null cases.

In [ ]:
calc.cutoff = function(p, B = 20, max=1){

  m <- length(p)
  m0 <- m
  bin <- c(-0.1, (1:B)/B*max)
  bin.counts=rep(0,B)

  for(i in 1:B){
    bin.counts[i]=sum((p>bin[i])&(p<=bin[i+1]))
  }

  tail.means <- rev(cumsum(rev(bin.counts))/(1:B))
  temp <- bin.counts - tail.means
  index <- min((1:B)[temp <= 0])
  cutoff2 <- (index)/B*max
  if(cutoff2 == 1) {cutoff2 <- 1-1/B}

  return(cutoff2)

}

cutoff_value1 = calc.cutoff(pvals1, B=20, max=1)
cutoff_value2 = calc.cutoff(pvals2, B=20, max=1)

cutoff = cbind(c(cutoff_value1), c(cutoff_value2))

colnames(cutoff) = c("cutoff_value1", "cutoff_value2")

# cutoff

**Filter dataset using cutoff points**

selecting pvalues in the upper-right quadrant using Histogram-Based method
defined as lambda1 = cutoffvalue1 and lambda2 = cutoffvalue2 for pvalue 1 and pvalue 2 respectively.
lambda1, lambda2 - probability of pvalue 1 and pvalue 2 in the upper-right quadrant respectively

In [ ]:
p_vals = pval_raw  %>% filter(pvalue >=cutoff_value1, limma_pvalue>=cutoff_value2)

convert selected pvalues to z values

In [ ]:
z_val = as.data.frame(qnorm(as.matrix(p_vals), lower.tail = TRUE))
colnames(z_val) = c("zvals1", "zvals2")

zvals1 <- (as.data.frame(as.matrix(z_val[,1])))
zvals2 <- (as.data.frame(as.matrix(z_val[,2])))

convert lambda(truncation points) to z values

In [ ]:
z_val_extremums = as.data.frame(qnorm(as.matrix(cbind(c(cutoff_value1,1),c(cutoff_value2,1))), lower.tail = TRUE))

min_z1 <- z_val_extremums[1,1]
min_z2 <- z_val_extremums[1,2]

**Estimating m00s**

m0.1 - DE genes in experiment 1
m0.2 - DE genes in experiment 2
m11 - DE genes in both experiments
m00 - EE genes in both experiments

In [ ]:
estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)
  m0.1 <- min(m0.1, 10000)

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)
  m0.2 <- min(m0.2, 10000)


  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  #pA <- (1-c1)*(1-c2)
  #m00 <- nA/pA
  
  #here, we used converted pvalues to z values to estimnate m00
  # density function for each row of the bivariate z values (x) and 
  # estimated parameters(rho)
  density = function(x, rho)
  {
    sigma = matrix(c(1, rho, rho, 1), 2, 2)
    z = dtmvnorm(x, mean = c(0,0), sigma = sigma, lower = c(min_z1, min_z2))
  }
  
  # log likelihood of the joint densities
  log_likelihood_fn = function(rho){
    
    joint_likelihood = z_val %>% split(.$zvals2) %>% map_dfr(~density(c(.$zvals1,.$zvals2),rho))    
    return(-sum(log(joint_likelihood)))
    
  }
  
  #MLE of the log likelihood function
  optimal_rho = optimize(log_likelihood_fn, lower = -1, upper = 1 )
  optimal_rho = as.data.frame(optimal_rho)
  
  #probability of a random variable greater than cutoff values 
  rho = as.numeric(optimal_rho[1])
  obj_value = optimal_rho[2]
  pA = pmvnorm(lower=c(min_z1, min_z2), upper=c(Inf, Inf), mean=c(0,0), sigma = matrix(c(1, rho, rho, 1), 2, 2))
  

  m00 <- nA/pA
  m00 <- min(m00, 10000)
  
  
  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  if (m0.1 == 10000 || m0.2 == 10000 || m00 == 10000){
    m11 = 0
  }
  
  ret <- list()
  ret$ms <- c(obj_value, rho, m, m0.1, m0.2, m00, m11)
  names(ret$ms) <- c("obj_value", "optimal_rho", "m", "m0.1", "m0.2", "m00", "m11")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

estimate.m0s(pvals1, pvals2, B=20)

$ms
$ms$obj_value
[1] -1084.877

$ms$optimal_rho
[1] 0.9949046

$ms$m
[1] 10000

$ms$m0.1
[1] 9336.667

$ms$m0.2
[1] 10000

$ms$m00
[1] 9336.667

$ms$m11
[1] 0


$cutoffs
[1] 0.70 0.05

# **Orr, 2012 Method**

In [ ]:
estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)
  m0.1 <- min(m0.1, 10000)
  

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)
  m0.2 <- min(m0.2, 10000)

  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  pA <- (1-c1)*(1-c2)
  m00 <- nA/pA
  m00 <- min(m00, 10000)


  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  if (m00 == 10000 || m0.1 == 10000 || m0.2 == 10000){
    m11 = 0
  }
  
  ret <- list()
  ret$ms <- c(m, m0.1, m0.2, m11, m00)
  names(ret$ms) <- c("m", "m0.1", "m0.2", "m11","m00")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

estimate.m0s(pvals1, pvals2, B=20)

$ms
        m      m0.1      m0.2       m11       m00 
10000.000  9336.667 10000.000     0.000  9828.070 

$cutoffs
[1] 0.70 0.05

# **Intersection Method**

In [ ]:
adj_pvalue_df <- data.frame(adj_pvalue)
# head(adj_pvalue_df)

In [ ]:
pvalue_p <- adj_pvalue_df[rowSums(adj_pvalue_df[1]<=0.05), ]
pvalue_05 <- length(pvalue_p)
#pvalue_05

In [ ]:
limma_p <- adj_limma_pvalue[rowSums(adj_limma_pvalue[1]<=0.05), ]
limma_05 <- length(limma_p) 
limma_05

[1] 0

In [ ]:
limma_ttest <- c(adj_pvalue_df, adj_limma_pvalue)
limma_ttest <- data.frame(limma_ttest)

In [ ]:
limma_ttest_p <- limma_ttest[rowSums((limma_ttest[1]<=0.05) & (limma_ttest[2]<=0.05)), ]
limma_ttest_05 <- nrow(limma_ttest_p)
limma_ttest_05

[1] 0

In [ ]:
pvalue_p2 <- adj_pvalue_df[rowSums(adj_pvalue_df[1]<=0.1), ]
pvalue_1 <- length(pvalue_p2)
pvalue_1

[1] 0

In [ ]:
limma_p2 <- adj_limma_pvalue[rowSums(adj_limma_pvalue[1]<=0.1), ]
limma_1 <- length(limma_p2) 
limma_1

In [ ]:
# write.csv(limma_ttest, "adj_pvalue_limma.csv", row.names=FALSE)

In [ ]:
limma_ttest_p2 <- limma_ttest[rowSums((limma_ttest[1]<=0.1) & (limma_ttest[2]<=0.1)), ]
limma_ttest_1 <- nrow(limma_ttest_p2)
limma_ttest_1

[1] 0

# **Venn diagram Analysis**

Controlling at alpha<=0.05 using Benjamini-Hochberg

In [ ]:
draw.pairwise.venn(area1=limma_1, area2=pvalue_1,cross.area=limma_ttest_1,
                   category=c("Student t-test","Limma"),fill=c("Red","Yellow"),
)

ERROR: ignored